In [1]:
import sqlite3
import pandas
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.sae_classicmodel

# Création de la connexion
conn = sqlite3.connect("ClassicModel.sqlite")

In [2]:
# 1 Lister les clients n’ayant jamais effecuté une commande

a=db.customers.aggregate([
    {
        "$lookup": {
            "from": "orders",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "customerOrders"
        }
    },
    {
        "$match": {
            "customerOrders": { "$size": 0 }
        }
    },
    {
        "$project": {
            "_id": 0,
            "customerNumber": 1,
            "customerName": 1,
            "contactFirstName": 1,
            "contactLastName": 1
        }
    }
])
pandas.DataFrame(list(a))

,customerNumber,customerName,contactLastName,contactFirstName
0,125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek
1,168,American Souvenirs Inc,Franco,Sue
2,169,Porto Imports Co.,de Castro,Isabel
3,206,"Asian Shopping Network, Co",Walker,Brydey
4,223,Natürlich Autos,Kloss,Horst
5,237,ANG Resellers,Camino,Alejandra
6,247,Messner Shopping Network,Messner,Renate
7,273,"Franken Gifts, Co",Franken,Peter
8,293,BG&E Collectables,Pon,Ed
9,303,Schuyler Imports,Schuyler,Bradley


In [34]:
#2 Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci

pipeline = [
    {
        # Étape 1 : Joindre les clients à la collection employees
        "$lookup": {
            "from": "customers",  # Joindre la collection "customers" pour obtenir les clients par employé
            "localField": "employeeNumber",  # Relier employeeNumber des employés
            "foreignField": "employee_id",  # Relier employee_id des clients
            "as": "employeeCustomers"  # Résultat de la jointure dans un tableau "employeeCustomers"
        }
    },
    {
        # Étape 2 : Grouper les informations par employé
        "$project": {
            "_id": 0,
            "employeeNumber": 1,
            "firstName": 1,
            "lastName": 1,
            "numberOfCustomers": { "$size": "$employeeCustomers" },  # Compter le nombre de clients associés à l'employé
            "numberOfOrders": {
                "$sum": {
                    "$size": { "$map": {
                        "input": "$employeeCustomers",  # Pour chaque client
                        "as": "customer",
                        "in": { "$size": { "$ifNull": ["$$customer.orders", []] } }  # Compter le nombre de commandes par client
                    }}
                }
            },
            "totalOrderAmount": {
                "$sum": {
                    "$sum": {
                        "$map": {
                            "input": "$employeeCustomers",  # Pour chaque client
                            "as": "customer",
                            "in": {
                                "$sum": {  # Calculer le montant total des commandes
                                    "$map": {
                                        "input": "$$customer.orders",  # Parcourir les commandes du client
                                        "as": "order",
                                        "in": {
                                            "$sum": {  # Calculer la somme des montants des produits commandés
                                                "$map": {
                                                    "input": "$$order.OrderDetails",
                                                    "as": "orderDetail",
                                                    "in": { "$multiply": ["$$orderDetail.quantityOrdered", "$$orderDetail.priceEach"] }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
]

# Exécution de l'agrégation sur la collection 'employees'
result = db.employees.aggregate(pipeline)

# Conversion des résultats en DataFrame
df = pd.DataFrame(list(result))
print(df)


    employeeNumber   lastName firstName  numberOfCustomers  numberOfOrders  \
0             1002     Murphy     Diane                  0               0   
1             1056  Patterson      Mary                  0               0   
2             1076   Firrelli      Jeff                  0               0   
3             1088  Patterson   William                  0               0   
4             1102     Bondur    Gerard                  0               0   
5             1143        Bow   Anthony                  0               0   
6             1165   Jennings    Leslie                  0               0   
7             1166   Thompson    Leslie                  0               0   
8             1188   Firrelli     Julie                  0               0   
9             1216  Patterson     Steve                  0               0   
10            1286      Tseng  Foon Yue                  0               0   
11            1323     Vanauf    George                  0      

In [30]:
#6 On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client;

pipeline2 = [
    # 1. Joindre la collection 'customers' pour obtenir le pays du client
    {
        "$lookup": {
            "from": "customers",  # Collection à joindre
            "localField": "customerNumber",  # Champ local dans 'orders'
            "foreignField": "customerNumber",  # Champ correspondant dans 'customers'
            "as": "customerInfo"  # Nom du tableau résultant
        }
    },
    {
        "$unwind": "$customerInfo"  # Décomposer 'customerInfo' pour accéder au pays du client
    },
    # 2. Filtrer les commandes qui ont des 'OrderDetails' non vides
    {
        "$match": {
            "OrderDetails": { "$exists": True, "$ne": [] }  # Assurez que 'orderDetails' existe et n'est pas vide
        }
    },
    # 3. Décomposer le tableau 'OrderDetails' pour chaque produit dans la commande
    {
        "$unwind": "$OrderDetails"  # Décomposer chaque commande en ligne de produits
    },
    # 4. Grouper les résultats par 'productLine' et 'country'
    {
        "$group": {
            "_id": {
                "productLine": "$OrderDetails.productLine",  # Correctement accéder à 'productLine'
                "country": "$customerInfo.country"  # Accéder au pays du client
            },
            "numberOfOrders": { "$sum": 1 }  # Comptabiliser le nombre de commandes pour chaque combinaison
        }
    },
    # 5. Projeter les résultats dans un format plus lisible
    {
        "$project": {
            "_id": 0,  # Ne pas afficher le champ '_id'
            "productLine": "$_id.productLine",  # Renommer '_id' en 'productLine'
            "country": "$_id.country",  # Renommer '_id' en 'country'
            "numberOfOrders": 1  # Garder le nombre de commandes
        }
    },
    # 6. Trier les résultats par 'productLine' et 'country'
    {
        "$sort": {
            "productLine": 1,  # Trier par 'productLine'
            "country": 1  # Puis trier par 'country'
        }
    }
]

# Exécution de l'agrégation sur la collection 'orders'
result = db.orders.aggregate(pipeline2)

# Conversion du résultat en DataFrame pour un affichage plus simple
df = pd.DataFrame(list(result))
print(df)


     numberOfOrders   productLine    country
0               212  Classic Cars  Australia
1               100  Classic Cars    Austria
2                16  Classic Cars    Belgium
3                56  Classic Cars     Canada
4               136  Classic Cars    Denmark
..              ...           ...        ...
121              56  Vintage Cars  Singapore
122             296  Vintage Cars      Spain
123              48  Vintage Cars     Sweden
124             156  Vintage Cars         UK
125             896  Vintage Cars        USA

[126 rows x 3 columns]
